# Baby Naming Trend

## Table of Content

- [Introduction](#introduction)
- [The Desire to be Different](#different_desire)
- [Celebrity Influence](#celebrity_influence)
- [Recent Trend](#recent_trend)
- [Conclusion](#conclusion)
- [Limitation & Future Research Direction](#limitation)
- [Appendix](#appendix)
- [Reference](#reference)

<a id='introduction'></a>

## Introduction

Ever wonder how many people in the US share the same name as you? The naming trend data from Social Security application might be a way to find out the answer. The data records the number of babies born with a particular name for each year and each state starting from 1910 and is available on Data.gov (Ref: 1). Only popular names (names with at least 5 babies born in the same year and same state) are included in the dataset.

The project is inspired by this dataset and I would like to look at what might affect how parents in the US name their babies and some recent naming trends through visualization.

Although the data from as early as 1910 is available, I've decided to focus on data from 1936 and onwards for more accurate trend since the Social Security Administration was established in August 14, 1935 and the data before that is very limited. (Please refer to the appendix for further details.)

<a id='different_desire'></a>

Let's first find out what are the factors that might influence how babies are named.

## Factors affecting how bab

## The Desire to be Different

I have a feeling that I am seeing more and more uncommon names (e.g. Honor, Cosmo) and I would like to verify if this is true or not. To complete my analysis I've incorporated a dataset from the US Department of Health and Human Services which records the number of births each year (Ref: 2). 

In [34]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [2]:
import os
os.chdir('/Users/arielyeung/Documents/Project/Data/Name_by_state')

In [3]:
df_full = pd.DataFrame()
for files in glob.glob("*.TXT"):
    df_full = pd.concat([df_full, pd.read_csv(files, header=None)], axis=0)

In [4]:
var_name = ['state', 'gender', 'year', 'name', 'count']

In [5]:
df_full.columns = var_name

In [6]:
# Only focus on the data after the Social Security Administration was established
df = df_full[df_full['year'] > 1935]

In [7]:
# Incorporate the birth data
df_birth = pd.read_csv('/Users/arielyeung/Documents/Project/Data/Birth_data.csv')

In [8]:
df_unique_name = df.groupby('year')['name'].nunique()

In [9]:
df_yearly = pd.DataFrame(df.groupby('year')['count'].sum()).reset_index()

In [10]:
df_yearly = pd.merge(df_yearly, df_birth)

For the number of unique popular name used (I use 'popular name' here because we only have the data for names with at least 5 babies named in the same year and same state from Social Security application), we are seeing an increasing trend and this increasing trend is not because more babies are born - for example, the number of births in the recent years is comparable to that during the baby boom in 1960s but the number of unique popular name used recently is more than double that in 1960s. Thus, nowadays people seem to have a stronger preference for variations.

In [11]:
df_yearly['unpop_count'] = df_yearly['birth_number'] - df_yearly['count']

People's growing desire to be different is also evident when comparing the total number of births and the number of babies born with popular names from the Social Security application data. The colored area represents the number of babies born with unpopular names and we can see that the area gets bigger and bigger over time. Convert this number of babies born with unpopular names to a standalone visualization we have:

This growing number of babies born with less popular names, together with the increasing number of unique popular names used we found earlier, indicate a stronger preference for uniqueness when parents choose a name for their baby.

<a id='celebrity_influence'></a>

## Celebrities' Influence

Next, I am interested to see whether celebrities can influence people's decisions in naming their babies. I pick some uncommon celebrity names trying to look at the relationship between the naming trend of these uncommon names and the celebrities' defining moments in their career.

For the names that are not included in the Social Security application dataset due to low usage (the number of babies named for a particular year/ state are less than 5), I assign 0 to these values for simpler visualization.

In [15]:
def create_graph_time(df, name, gender, event, event_year, event2, event_year2, event_position2):
    df_year_pad = pd.DataFrame(np.unique(df['year'])).rename(columns={0: 'year'})
    df_name = pd.DataFrame(df[df['name'] == name].groupby('year')['count'].sum()).reset_index()
    df_name_pad = pd.merge(df_year_pad, df_name, how='outer').fillna(0)
    color_tone = 'rgb(255, 153, 204)' if gender == 'female' else 'rgb(102, 178, 255)'
    graph = go.Scatter(x=df_name_pad['year'], y=df_name_pad['count'],
                       line=dict(color=color_tone, width=3), fill='tonexty')
    line = go.Scatter(x=[event_year]*df_name_pad['count'].max().astype(int),
                      y=[i for i in range(df_name_pad['count'].max().astype(int))],
                      line=dict(color=color_tone, width=3, dash='dot'),
                      hoverinfo='none')
    data = [graph, line]
    layout = dict(title='Baby Naming Trend for '+name, titlefont=dict(size=22),
                  xaxis=dict(rangeslider=dict(visible=True),
                             title='Year', titlefont=dict(size=16), tickfont=dict(size=13)),
                  yaxis=dict(title='No. of babies named',
                             titlefont=dict(size=16), tickfont=dict(size=13)),
                  annotations=[dict(x=event_year, y=df_name_pad['count'].max().astype(int),
                                    xref='x', yref='y', text=event, ax=0, ay=-40,
                                    font=dict(size=16)),
                               dict(x=event_year2, y=int(event_position2),
                                    xref='x', yref='y', text=event2, ax=0, ay=25,
                                    font=dict(size=16))],
                  showlegend=False)
    fig = dict(data=data, layout=layout)
    return fig

### Rihanna

We definitely see some influence from celebrity here. Before the singer Rihanna released her first single in 2005, almost no one named their kid Rihanna. After her debut, more and more babies were born with that name and the number peaked at 2008, the same year when the singer earned her first Grammy Award for her song "Umbrella" (Ref: 3, 4). However, the increasing trend did not last and recently, only around 100 people named their kid Rihanna.

I am curious if this celebrity influence would differ by region. I look at the percentage of babies named Rihanna per state instead of the raw number for an apple to apple comparison. For time period, I focus on year 2008 since it is when the number of babies named Rihanna peaked.

In [12]:
def create_choromap(df, name, gender, event_year):
    gen = 'F' if gender == 'female' else 'M'
    df_name_filter = df[(df['name'] == name) & (df['gender'] == gen) & (df['year'] == event_year)]
    df_name_state = pd.DataFrame(df_name_filter.groupby('state')['count'].sum()).reset_index()
    df_name_state = df_name_state.rename(columns={'count': 'name_count'})
    df_filter = df[(df['year'] == event_year) & (df['gender'] == gen)]
    df_all_name_state = pd.DataFrame(df_filter.groupby('state')['count'].sum()).reset_index()
    df_all_name_state = df_all_name_state.rename(columns={'count': 'all_count'})
    df_name_state = pd.merge(df_name_state, df_all_name_state, how='outer')
    df_name_state['name_pct'] = round(df_name_state['name_count'] / df_name_state['all_count'] * 100, 3)
    df_name_state['name_pct'] = df_name_state['name_pct'].fillna(0)
    scl_female = [[0.0, 'rgb(224, 224, 224)'], [0.25, 'rgb(255, 204, 229)'],
                  [0.5, 'rgb(255, 102, 178)'], [0.75, 'rgb(255, 0, 127)'],
                  [1.0, 'rgb(102, 0, 51)']]
    scl_male = [[0.0, 'rgb(224, 224, 224)'], [0.25, 'rgb(204, 229, 255)'],
                [0.5, 'rgb(51, 153, 255)'], [0.75, 'rgb(0, 0, 255)'],
                [1.0, 'rgb(0, 0, 102)']]
    scale = scl_female if gender == 'female' else scl_male
    data = [dict(type='choropleth', colorscale=scale, autocolorscale=False,
                 locations=df_name_state['state'], z=df_name_state['name_pct'],
                 locationmode='USA-states',
                 marker=dict(line=dict(color='rgb(255,255,255)', width=2)),
                 colorbar=dict(title='% Named', thickness=15, len=0.6,
                               tickfont=dict(size=14), titlefont=dict(size=14)))]
    layout = dict(title='Percentage of Babies Named ' + name + ' by State in ' + str(event_year),
                  font=dict(size=14),
                  geo=dict(scope='usa', projection=dict(type='albers usa')))
    fig = dict(data=data, layout=layout)
    return fig

In [35]:
choromap_rihanna = create_choromap(df, 'Rihanna', 'female', 2008)
py.iplot(choromap_rihanna, filename='choromap_rihanna')

In [30]:
df.head()

,state,gender,year,name,count
10640,IN,F,1936,Mary,1370
10641,IN,F,1936,Shirley,1115
10642,IN,F,1936,Barbara,881
10643,IN,F,1936,Patricia,785
10644,IN,F,1936,Nancy,649


In [32]:
df_name_filter = df[(df['name'] == 'Rihanna') & (df['gender'] == 'F') & (df['year'] == 2008)]
df_name_state = pd.DataFrame(df_name_filter.groupby('state')['count'].sum()).reset_index()
df_name_state = df_name_state.rename(columns={'count': 'name_count'})
df_filter = df[(df['year'] == 2008) & (df['gender'] == 'F')]
df_all_name_state = pd.DataFrame(df_filter.groupby('state')['count'].sum()).reset_index()
df_all_name_state = df_all_name_state.rename(columns={'count': 'all_count'})
df_name_state = pd.merge(df_name_state, df_all_name_state, how='outer')
df_name_state['name_pct'] = round(df_name_state['name_count'] / df_name_state['all_count'] * 100, 3)
df_name_state['name_pct'] = df_name_state['name_pct'].fillna(0)
scale = [[0.0, 'rgb(224, 224, 224)'], [0.25, 'rgb(255, 204, 229)'],
                  [0.5, 'rgb(255, 102, 178)'], [0.75, 'rgb(255, 0, 127)'],
                  [1.0, 'rgb(102, 0, 51)']]
data = [dict(type='choropleth', colorscale=scale, autocolorscale=False,
                 locations=df_name_state['state'], z=df_name_state['name_pct'],
                 locationmode='USA-states',
                 marker=dict(line=dict(color='rgb(255,255,255)', width=2)),
                 colorbar=dict(title='% Named', thickness=15, len=0.6,
                               tickfont=dict(size=14), titlefont=dict(size=14)))]
layout = dict(title='Percentage of Babies Named Rihanna by State in 2008',
                  font=dict(size=14),
                  geo=dict(scope='usa', projection=dict(type='albers usa')))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='choromap_rihanna')

In 2008, Rihanna influenced different states differently and it seems like she had a slightly greater impact in the southeastern part of US. The state with the highest percentage is actually DC but it's too small to be shown here.

### Beyonce

I also look into other unique artist name to see if similar trend exists.

We see a similar surge for the name Beyonce after the singer released her first single in 1997 as Destiny's Child although the number of baby named is smaller than that of Rihanna's (Ref: 5). The number reached its maximum in 2001 but dropped drastically afterwards. There was a small spike in 2003 after Beyonce released her first solo single, as well as in 2007, the year after what is believed to be Beyonce's biggest hit - "Irreplaceable" came out. Yet, less and less people used the name after that and barely anyone named their kid Beyonce in the recent years.

Looking at the choropleth map for the percentage of babies named Beyonce in 2001, Mississippi dominated and I am surprised that it wasn't Texas since Beyonce used to lived in Texas. Plus, there were quite a lot of states with no babies named Beyonce.

### Kobe

Other than singer/ artist, I also explore the influence from famous people in other areas such as sports and politics. I pick Kobe since people are saying Kobe Bryant's parents named him after seeing a japanese kobe steak in a restaurant menu prior to his birth and it would be interesting to see if people actually follow the trend and name their kids after a steak.

We are seeing a similar pattern - people began using the name after the popular basketball player Kobe Bryant's first NBA game. In 2000-2002, a lot of babies were named Kobe and this is not surprising as Kobe Bryant led his team LA Lakers to NBA championship during 2000-2002. Yet, the number dropped significantly in 2004 when the NBA star was accused of sexual assault in 2003-2004 and not a lot of babies were named Kobe since then. There was a slight increase in 2008 when the player won NBA MVP award and it lasted for a bit when Lakers won back-to-back NBA championships in 2009-2010. Nowadays, there are still around 500 babies named Kobe each year.

Kobe had the greatest influence in Montana in 2001. Again, I am surprised that only a small number of babies were named Kobe in California - where Lakers' home city is at.

### Barack

For politician, I try Barack, which is the first name of President Obama. There was a spike in 2008-2009 for the number of babies named Barack when President Obama was running for presidency and got elected although the number was very small. I am surprised by the fact that not a lot of parents named their babies Barack and also the trend didn't last. I am guessing that's because we seldom refer to our presidents with their first name.

In 2009, the only states having babies born with the name Barack according to the Social Security application data were Michigan, New York, California and Texas. At first I was a bit surprised that Texas was one of them since it is a republican state, but we should keep in mind that these states only added up to 27 in total, which is a very small number.

<a id='recent_trend'></a>

I am also interested in finding out the most popular baby names in the past decade and look at their trends, as well as regional differences in usage.

## Recent Trend

In [17]:
df_female = pd.DataFrame(df[df['gender'] == 'F'].
                         groupby(['year', 'name'])['count'].sum()).reset_index()
df_female_post2007 = pd.DataFrame(df_female[df_female['year'] >= 2007].
                                  groupby('name')['count'].sum()).reset_index()

The most popular names for baby girls over the past decade are Sophia and Emma.

### Sophia

Looking at the trend of Sophia, it was not widely used back in the 90s but has become more and more popular since then. It peaked in 2012 but we are seeing a decreasing trend after that.

From Sophia's choropleth map in 2012, people in Rhode Island really liked the name, followed by several other states including Vermont, New Hampshire, Wyoming and Hawaii.

### Emma

Emma has gained its popularity since the 90s and is still very popular nowadays.

Emma was a popular name choice in North Dakota, Maine, Vermont and New Hampshire in 2003 but was not popular in California, New Mexico and Texas.

In [31]:
df_male = pd.DataFrame(df[df['gender'] == 'M'].
                       groupby(['year', 'name'])['count'].sum()).reset_index()
df_male_post2010 = pd.DataFrame(df_male[df_male['year'] > 2010].
                                groupby('name')['count'].sum()).reset_index()

The most popular names for baby boys over the past decade are Mason and Noah.

Similar to Sophia, people started to use Mason more and more since around the 90s. The number registered a maximum in 2011 but gradually declined after that.

The choropleth map of Mason is actually pretty similar to that of Emma - both were frequently used in North Dakota, Maine, Vermont but not in California, New Mexico and Texas. Unlike Emma, Mason was also used a lot in Rhode Island and South Dakota.

Similar to other popular names in the recent decade, Noah also started gaining its popularity since the 90s. It quickly reached the zenith in 1999 and still remains popular nowadays.

In 2015, Noah is favored in Rhode Island, Delaware, West Virginia and Hawaii.

<a id='conclusion'></a>

## Conclusion

Some factors that might influence people's decision in naming their babies are explored and there's evidence that parents nowadays prefer more unqiue names and might also be influenced by celebrities although the degree of influence varies by state. We also find out the names that top the list of most frequently used names in the past decade and they are Sophia, Emma, Mason and Noah.

<a id='limitation'></a>

## Limitation & Future Research Direction

- This analysis is limited by the fact that only the data for names with at least 5 babies born in the same year and same state are published and it would be helpful if I can get the full set of data for a more accurate picture of the trends and regional differences.

- It would also be interesting to gather some data from social media such as celebrities' no. of Facebook posts/ likes and no. of tweets and look at how that correlates to naming trend.

<a id='appendix'></a>

## Appendix

When comparing the number of births and the number of babies applying for Social Security, the big gap between the two from 1910s to 1930s is probably because the Social Security Administration was established in August 14, 1935 and not that many people applied for Social Security before that. Thus, data from Social Security application before 1936 would underestimate the actual number of babies born with a particular name and therefore, I've decided to only focus on the data starting from 1936.

<a id='reference'></a>

## Reference

(1) https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-data-by-state-and-district-of- 

(2) https://catalog.data.gov/dataset/births-and-general-fertility-rates-united-states-1909-2013/resource/a4d978b0-3396-4bb9-b967-b3c8c69b03a3?inner_span=True

(3) https://www.biography.com/people/rihanna-201257

(4) https://en.wikipedia.org/wiki/List_of_awards_and_nominations_received_by_Rihanna

(5) https://worldhistoryproject.org/topics/beyonce-knowles/page/1

(6) https://www.cnn.com/2013/04/29/us/kobe-bryant-fast-facts/index.html

